# Remote Diplomacy Game Engine: Quickstart

A persistent, remote game engine is hosted at shade.tacc.utexas.edu:8432. The accompanying Web UI can be accessed on port 3000. The Diplomacy python client and Web UI are the primary modes to interact with the engine. While it is expected to be highly available, unexpected center-wide network issues and planned maintainence are possibilities. The quickstart demonstrates how to connect to the game engine, create a game, and launch seven independent processes to play through a Diplomacy game by submitting random orders. The `ray` package and Ray actor classes are used to create these independent players. See the Network Game section of the README file at https://github.com/SHADE-AI/diplomacy for an example script run via Python interpreter (Jupyter notebooks rely on an asyncio event loop which doesn't jive with the example in github). Be sure to replace the hostname and port.

In [ ]:
import asyncio
import random
import diplomacy.client
from diplomacy.client.connection import connect
from diplomacy.utils import exceptions
import ray

In [ ]:
ray.init()

In [ ]:
@ray.remote
class RemotePlayer:
    
    def __init__(self,username, password, power, hostname, port):
        self.game_id = ""
        self.power = power
        self.username = username
        self.password = password
        self.hostname = hostname
        self.port = port
        self.game = None
        self.channel = None
        self.connect = None
    
    async def join_game(self,game_id):
        self.game_id = game_id
        print("user: {}\tgame_id: {}\tpower: {}\thost: {}\tport: {}".format(self.username,self.game_id, self.power,self.hostname,self.port))
        
        self.connection = await connect(self.hostname, self.port)
        self.channel = await self.connection.authenticate(self.username, self.password)
        # Waiting for the game, then joining it
        while not (await self.channel.list_games(game_id=self.game_id)):
            await asyncio.sleep(1.)
        self.game = await self.channel.join_game(game_id=self.game_id, power_name=self.power)
    
    async def play(self):
        if self.game == None:
            return
        while not self.game.is_game_done:
            current_phase = self.game.get_current_phase()
            
            #submit random, legitimate orders
            if self.game.get_orderable_locations(self.power):
                possible_orders = self.game.get_all_possible_orders()
                orders = [random.choice(possible_orders[loc]) for loc in self.game.get_orderable_locations(self.power) if possible_orders[loc]]
                print('[%s/%s] - Submitted: %s' % (self.power, self.game.get_current_phase(), orders))
                await self.game.set_orders(power_name=self.power, orders=orders, wait=False)
            

            while current_phase == self.game.get_current_phase():
                await asyncio.sleep(0.1)

        
    

In [ ]:
POWERS = ['AUSTRIA', 'ENGLAND', 'FRANCE', 'GERMANY', 'ITALY', 'RUSSIA', 'TURKEY']
host = "shade.tacc.utexas.edu"
port = 8432

### Connect to engine and create game

In [ ]:
#default - creating game without joining as a power returns an omniscient observer
connection = await connect(host,8432)

#connection.authenticate will create user if not already exists
channel = await connection.authenticate('game_master','password')

game_id = "Test_game5"
game_master = await channel.create_game(game_id=game_id, rules={'REAL_TIME', 'NO_DEADLINE', 'POWER_CHOICE'})

### Instantiate RemotePlayer objects and join game

In [ ]:
players = [RemotePlayer.remote("user_"+pow, "password", pow, host,port) for pow in POWERS]
ray.get([p.join_game.remote(game_id) for p in players])

### Instruct each RemotePlayer to begin playing (random orders)
Note: errors related to web socket connection thrown when game finishes. Need to look into this further.

In [ ]:
ray.get([p.play.remote() for p in players])